# CCoHG

In [ ]:
import random
import math
import networkx as nx
from torch_geometric.utils.convert import from_networkx
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def hamiltonian(n, p=0.5):
    edge_list = []
    for i in range(n-1):
        edge_list.append((i,i+1))
    edge_list.append((0, n-1))
    for i in range(n):
        for j in range(n):
            if j>i and abs(i-j)>1 and abs(i-j)<n-1 and random.random() < p:
                edge_list.append((i,j))
    return edge_list

In [ ]:
def yielder(_edge_list, index, edges):
    if index<len(edges):
        i,j = edges[index]
        yield from yielder(_edge_list, index+1, edges)
        yield from yielder(_edge_list+[(i,j)], index+1, edges)
    else:
        yield _edge_list

def all_hamiltonians(n):
    edge_list = []
    for i in range(n-1):
        edge_list.append((i,i+1))
    edge_list.append((0, n-1))
    edges = [(i,j) for i in range(n) for j in range(n) if j>i and abs(i-j)>1 and abs(i-j)<n-1]
    random.shuffle(edges)
    yield from yielder(edge_list, 0, edges)

In [ ]:
def CCoHG(edge_list, n):
    modified_edge_list = []
    for (i, j) in edge_list:
        modified_edge_list.append((i,j))
        modified_edge_list.append((j,i))
        modified_edge_list.append((i+n,j+n))
        modified_edge_list.append((j+n,i+n))
        modified_edge_list.append((i+(2*n),j+(2*n)))
        modified_edge_list.append((j+(2*n),i+(2*n)))
        modified_edge_list.append((i+(3*n),j+(3*n)))
        modified_edge_list.append((j+(3*n),i+(3*n)))
        if (i==0 and j==n-1):
            modified_edge_list.append((i,j+(3*n)))
            modified_edge_list.append((j+(3*n), i))
            modified_edge_list.append((i+(3*n),j))
            modified_edge_list.append((j, i+(3*n)))
            modified_edge_list.append((i+n,j+(2*n)))
            modified_edge_list.append((j+(2*n),i+n))
            modified_edge_list.append((i+(2*n),j+n))
            modified_edge_list.append((j+n,i+(2*n)))
        else: #Its actually not important how exactly the following pairs are determined, however, if this is done differently, then the edge have to be matched
            modified_edge_list.append((i,j+n))
            modified_edge_list.append((j+n,i))
            modified_edge_list.append((i+n,j))
            modified_edge_list.append((j,i+n))
            modified_edge_list.append((i+(2*n),j+(3*n)))
            modified_edge_list.append((j+(3*n),i+(2*n)))
            modified_edge_list.append((i+(3*n),j+(2*n)))
            modified_edge_list.append((j+(2*n),i+(3*n)))
    return modified_edge_list

def CCoHG_pair(edge_list, n):
    edge_list = CCoHG(hamiltonian(n), n)
    nx_graph_1 = nx.from_edgelist(edge_list)
    nx_graph_2 = nx.from_edgelist(edge_list)
    nodes_1 = dict()
    for i in range(n):
        nodes_1[i] = 1
        nodes_1[i+n] = 0
        nodes_1[i+2*n] = 1
        nodes_1[i+3*n] = 0
    nodes_2 = dict()
    for i in range(n):
        nodes_2[i] = int(i<n/2)
        nodes_2[i+n] = int(i>=n/2)
        nodes_2[i+2*n] = int(i<n/2)
        nodes_2[i+3*n] = int(i>=n/2)
    nx.set_node_attributes(nx_graph_1, nodes_1, name="x")
    nx.set_node_attributes(nx_graph_2, nodes_2, name="x")
    return nx_graph_1, nx_graph_2

In [ ]:
hamiltonians = []
for n in range(3,8):
    hamiltonians_current = []
    for edge_list in all_hamiltonians(n):
        graph = nx.from_edgelist(edge_list)
        if max([b for a,b in graph.degree]) > 6:
            continue
        if not True in [nx.is_isomorphic(graph, graph2) for graph2 in hamiltonians_current]:
            hamiltonians_current.append(graph)
    hamiltonians.extend(hamiltonians_current)

In [ ]:
len(hamiltonians)

In [ ]:
hamiltonians_BREC = hamiltonians
while len(hamiltonians_BREC)>100:
    max_avg_degree = max([sum([b for a,b in graph.degree])/graph.order() for graph in hamiltonians_BREC])
    highs = [graph for graph in hamiltonians_BREC if sum([b for a,b in graph.degree])/graph.order()==max_avg_degree]
    lows = [graph for graph in hamiltonians_BREC if sum([b for a,b in graph.degree])/graph.order()<max_avg_degree]
    print(len(highs), len(lows))
    random.shuffle(highs)
    hamiltonians_BREC = lows + highs[:100-len(lows)]
    print(len(hamiltonians_BREC))

In [ ]:
with open("hamiltoniansBREC.graphml", "wb") as f:
    for graph in hamiltonians_BREC:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)

In [ ]:
if False:
    hamiltonians_BREC = []
    with open("hamiltoniansBREC.graphml", "rb") as f:
        while True:
            try:
                hamiltonians_BREC.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break

In [ ]:
CCoHG_graphs = []
for graph in hamiltonians_BREC:
    edges = graph.edges
    edges = [(int(a), int(b)) for a,b in edges]
    graph_1, graph_2 = CCoHG_pair(edges, graph.order())
    CCoHG_graphs.append(graph_1)
    CCoHG_graphs.append(graph_2)
with open("CCoHG_BREC.graphml", "wb") as f:
    for graph in CCoHG_graphs:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)
if False:
    CCoHG_graphs = []
    with open("CCoHG_BREC.graphml", "rb") as f:
        while True:
            try:
                CCoHG_graphs.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break
    print(len(CCoHG_graphs))
for graph in CCoHG_graphs:
    nx.draw_networkx(graph, with_labels=True, node_color=[["green", "red"][value] for key, value in nx.get_node_attributes(graph, name="x").items()])
    plt.show()
    plt.clf()

In [ ]:
print(len(hamiltonians))

## CCoHG Standard Dataset

In [ ]:
for n in range(8, 13):
    hamiltonians_current = []
    for edge_list in all_hamiltonians(n):
        if len(hamiltonians_current) >= 120:
            break
        graph = nx.from_edgelist(edge_list)
        if max([b for a,b in graph.degree]) > 6:
            continue
        if not True in [nx.faster_could_be_isomorphic(graph, graph2) for graph2 in hamiltonians_current]:#is_isomorphic
            hamiltonians_current.append(graph)
            print(n, len(hamiltonians_current), end="\r")
    hamiltonians.extend(hamiltonians_current)
    print(" "*50)

In [ ]:
print(len(hamiltonians))

In [ ]:
hamiltonians
while len(hamiltonians)>1000:
    max_avg_degree = max([sum([b for a,b in graph.degree])/graph.order() for graph in hamiltonians])
    highs = [graph for graph in hamiltonians if sum([b for a,b in graph.degree])/graph.order()==max_avg_degree]
    lows = [graph for graph in hamiltonians if sum([b for a,b in graph.degree])/graph.order()<max_avg_degree]
    print(len(highs), len(lows))
    random.shuffle(highs)
    hamiltonians = lows + highs[:1000-len(lows)]
    print(len(hamiltonians))

In [ ]:
with open("hamiltonians.graphml", "wb") as f:
    for graph in hamiltonians:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)

In [ ]:
if False:
    hamiltonians = []
    with open("hamiltonians.graphml", "rb") as f:
        while True:
            try:
                hamiltonians.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break

In [ ]:
CCoHG_graphs = []
for graph in hamiltonians:
    edges = graph.edges
    edges = [(int(a), int(b)) for a,b in edges]
    graph_1, graph_2 = CCoHG_pair(edges, graph.order())
    CCoHG_graphs.append(graph_1)
    CCoHG_graphs.append(graph_2)
with open("CCoHG.graphml", "wb") as f:
    for graph in CCoHG_graphs:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)
if False:
    CCoHG_graphs = []
    with open("CCoHG.graphml", "rb") as f:
        while True:
            try:
                CCoHG_graphs.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break
    print(len(CCoHG_graphs))
for graph in CCoHG_graphs:
    nx.draw_networkx(graph, with_labels=True, node_color=[["green", "red"][value] for key, value in nx.get_node_attributes(graph, name="x").items()])
    plt.show()
    plt.clf()

# 3 Regular 2 Regular

In [ ]:
import torch, torch_geometric
import matplotlib.pyplot as plt
import networkx as nx


from torch_geometric.data import Dataset, Data

pattern6 = nx.cycle_graph(6)

def count_pattern(G, pattern):
    GM = nx.algorithms.isomorphism.GraphMatcher(G, pattern)
    unique_p = set()

    for mapping in GM.subgraph_isomorphisms_iter():
        nodes_in_subgraph = frozenset(mapping.items())
        unique_p.add(nodes_in_subgraph)

    return len(unique_p)

def generate_3_regular_graph(N, min_diam=1):
    G = nx.random_regular_graph(3, N)
    while (not nx.is_connected(G)) or (nx.diameter(G) < min_diam):
        G = nx.random_regular_graph(3, N)

    return G

def split_edges(G, s=2):
    new_G = nx.Graph()

    for u in G.nodes():
      new_G.add_node(u)

    for u, v in G.edges():
        new_nodes = [u]
        for _ in range(s-1):
            new_node = max(new_G.nodes)+1
            new_nodes.append(new_node)
            new_G.add_node(new_node)
        new_nodes.append(v)
        for i in range(len(new_nodes)-1):
            new_G.add_edge(new_nodes[i], new_nodes[i+1])
            new_G.add_edge(new_nodes[i+1], new_nodes[i])

    return new_G


def networkx_to_pyg(G):
    nodes = list(G.nodes())
    node_idx = {node: i for i, node in enumerate(nodes)}

    edges = []
    for u, v in G.edges():
        edges.append((node_idx[u], node_idx[v]))
        edges.append((node_idx[v], node_idx[u]))

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    x = torch.ones((len(nodes), 1), dtype=torch.float)

    data = Data(x=x, edge_index=edge_index)
    data.diameter = nx.diameter(G)
    data.count_hex = count_pattern(G, pattern6)
    return data

N_min = 6
min_diam = 1

max_tries = 1000

In [ ]:
dataset = []
for c, s, N_max in [(2, 7, 12), (3, 5, 14), (4, 4, 20)]:
    print(c, s, N_max)
    for N in range(N_min, N_max, 2):
        basics = []
        counter = 0
        for _ in range(30):
            print(N, _)
            G = generate_3_regular_graph(N, min_diam)
            while True in [nx.is_isomorphic(G, _G) for _G in basics] or True in [len(cycle)>2 for cycle in nx.simple_cycles(G, length_bound=c)]:
                G = generate_3_regular_graph(N, min_diam)
                counter+=1
                if counter>max_tries:
                    break
            if counter>max_tries:
                break
            basics.append(G)
            #nx.draw(G)
            #plt.show()
            #plt.clf()
        print(len(basics))
        for i in range(len(basics)-1):
            for j in range(i+1, len(basics)):
                G = basics[i]
                G2 = basics[j]
                G_split = split_edges(G, s=s)
                G2_split = split_edges(G2, s=s)
                dataset.append((G_split, G2_split))
        print(len(dataset))

In [ ]:
import random
random.shuffle(dataset)

In [ ]:
numtolist = dict()
for pair in dataset:
    num_nodes = pair[0].number_of_nodes()
    if num_nodes in numtolist:
        numtolist[num_nodes].append(pair)
    else:
        numtolist[num_nodes]= [pair]
for key, value in numtolist.items():
    print(key, len(value))
dataset = []
for key, value in numtolist.items():
    dataset += value[:24]
print(len(dataset))

In [ ]:
_dataset = dataset
dataset = []
for G1, G2 in _dataset:
    dataset.append(G1)
    dataset.append(G2)

In [ ]:
import pickle
with open("3reg2reg.graphml", "wb") as f:
    for graph in dataset:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)

In [ ]:
if True:
    _3reg2reg = []
    with open("3reg2reg.graphml", "rb") as f:
        while True:
            try:
                _3reg2reg.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break
    for graph in _3reg2reg:
        nx.draw(graph)
        plt.show()
        plt.clf()

## 3r2r Standard Dataset

In [ ]:
import time
max_time = 60
dataset = [] # 10, 
for c, s, N_max, max_tries in [(2, 7, 22, 0), (3, 5, 30, 100), (4, 4, 38, 100)]:
    print(c, s, N_max)
    for N in range(N_min, N_max, 2):
        start = time.time()
        basics = []
        for _ in range(334):
            G = generate_3_regular_graph(N, min_diam)
            while True in [nx.is_isomorphic(G, _G) for _G in basics] or True in [len(cycle)>2 for cycle in nx.simple_cycles(G, length_bound=c)]:
                G = generate_3_regular_graph(N, min_diam)
                if time.time()-start > max_time:
                    break
            if time.time()-start > max_time:
                break
            basics.append(G)
            print(N, _)
            #nx.draw(G)
            #plt.show()
            #plt.clf()
        print(len(basics))
        for graph in basics:
                graph_split = split_edges(graph, s=s)
                dataset.append(graph_split)
        print(len(dataset))

In [ ]:
import random
random.shuffle(dataset)
numtolist = dict()
for graph in dataset:
    num_nodes = graph.number_of_nodes()
    if num_nodes in numtolist:
        numtolist[num_nodes].append(graph)
    else:
        numtolist[num_nodes]= [graph]
for key, value in sorted(numtolist.items(), key=lambda t: t[0]):
    print(key, len(value))
dataset = []
for key, value in sorted(numtolist.items(), key=lambda t: t[0]):
    dataset += value[:1000-len(dataset)]
print(len(dataset))

In [ ]:
import pickle
with open("3reg2reg1000.graphml", "wb") as f:
    for graph in dataset:
        gml = nx.generate_graphml(graph)
        pickle.dump(chr(10).join(gml), f)

In [ ]:
if True:
    _3reg2reg = []
    with open("3reg2reg1000.graphml", "rb") as f:
        while True:
            try:
                _3reg2reg.append(nx.parse_graphml(pickle.load(f)))
            except EOFError:
                break
    for graph in _3reg2reg:
        nx.draw(graph)
        plt.show()
        plt.clf()